# Pytorch

## Instalaciones

In [1]:
!pip install torch
!pip install torchvision

## Importar dependencias

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Definir Hiperparámetros

In [3]:
input_size = 784
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 1e-3

## Descargando la base de datos mnist

In [4]:
train_data = dsets.FashionMNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.FashionMNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:00<00:00, 45489318.84it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5407070.65it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 22512632.48it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6856867.89it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



## Leyendo la data

In [5]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

## Definir modelo

In [6]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

## Instancia del modelo

In [7]:
net = Net(input_size, hidden_size, num_classes)

if torch.cuda.is_available():
  net.cuda()

## Compilación

In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

## Entrenamiento

In [9]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))


Epoca [1/5], Step [100/600], Loss: 0.7143
Epoca [1/5], Step [200/600], Loss: 0.6362
Epoca [1/5], Step [300/600], Loss: 0.4887
Epoca [1/5], Step [400/600], Loss: 0.4926
Epoca [1/5], Step [500/600], Loss: 0.4979
Epoca [1/5], Step [600/600], Loss: 0.4410
Epoca [2/5], Step [100/600], Loss: 0.4400
Epoca [2/5], Step [200/600], Loss: 0.3632
Epoca [2/5], Step [300/600], Loss: 0.4335
Epoca [2/5], Step [400/600], Loss: 0.3104
Epoca [2/5], Step [500/600], Loss: 0.4282
Epoca [2/5], Step [600/600], Loss: 0.3711
Epoca [3/5], Step [100/600], Loss: 0.3998
Epoca [3/5], Step [200/600], Loss: 0.3388
Epoca [3/5], Step [300/600], Loss: 0.4127
Epoca [3/5], Step [400/600], Loss: 0.5255
Epoca [3/5], Step [500/600], Loss: 0.2856
Epoca [3/5], Step [600/600], Loss: 0.5201
Epoca [4/5], Step [100/600], Loss: 0.2846
Epoca [4/5], Step [200/600], Loss: 0.2719
Epoca [4/5], Step [300/600], Loss: 0.3372
Epoca [4/5], Step [400/600], Loss: 0.4646
Epoca [4/5], Step [500/600], Loss: 0.2771
Epoca [4/5], Step [600/600], Loss:

In [10]:
correct = 0
total = 0
for images, labels in test_gen:
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    output = net(images)
    _, predicted = torch.max(output, 1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

print('Accuracy: %.3f %%' % (100 * correct / (total + 1)))

Accuracy: 86.551 %
